In [1]:
from imprint.nb_util import setup_nb
setup_nb()
import confirm.adagrid as ada
import imprint as ip
from imprint.models.ztest import ZTest1D

g = ip.cartesian_grid(theta_min=[-1], theta_max=[1], null_hypos=[ip.hypo("x0 < 0")])
db = ada.ada_calibrate(
    ZTest1D,
    g=g,
    init_K=1024,
    nB=2,
    grid_target=0.005,
    bias_target=0.005,
    std_target=0.005,
    prod=False,
    tile_batch_size=1,
    coordinate_every=1,
    n_zones=2
)

[worker_id=None] 2023-03-15 15:35:28,141 - imprint - DEBUG 
Enabling 64-bit floats in JAX.
[worker_id=None] 2023-03-15 15:35:28,322 - imprint.grid - DEBUG 
_gen_short_uuids(n=2, worker_id=1, t=1678908928, n_bits=18, worker_bits=18) = [4693278577301651456 4693278577301651457, ...]:
[worker_id=1] 2023-03-15 15:35:28,419 - confirm.adagrid.init - DEBUG 
Initialized database with 1 tiles and 1 null hypos. The tiles are split between 2 zones with packet_size=1024.
[worker_id=1] 2023-03-15 15:35:28,427 - confirm.adagrid.backend - DEBUG 
init took 0.071 seconds
[worker_id=1] 2023-03-15 15:35:28,428 - confirm.adagrid.backend - DEBUG 
setup took 0.000 seconds
[worker_id=1] 2023-03-15 15:35:28,428 - confirm.adagrid.step - DEBUG 
waiting for work: (zone_id=0, step_id=0, packet_id=0)
[worker_id=1] 2023-03-15 15:35:28,431 - confirm.adagrid.step - DEBUG 
Processing 1 tiles.
[worker_id=1] 2023-03-15 15:35:28,622 - confirm.adagrid.step - DEBUG 
Processed 1 tiles in 0.19 seconds.
[worker_id=1] 2023-03-1

In [2]:
df = db.get_results()

In [3]:
import confirm.cloud.clickhouse as ch
chdb = ch.Clickhouse.connect()

[worker_id=None] 2023-03-15 15:35:31,796 - confirm.cloud.clickhouse - INFO 
Clickhouse config: default@t2n4z83v82.us-east-1.aws.clickhouse.cloud:8443/None
[worker_id=None] 2023-03-15 15:35:32,195 - confirm.cloud.clickhouse - DEBUG 
Command took 0.1785106658935547 seconds
create database if not exists c0adc07c960a4a809dedf9310f8a0363
[worker_id=None] 2023-03-15 15:35:32,196 - confirm.cloud.clickhouse - INFO 
Connected to job c0adc07c960a4a809dedf9310f8a0363


In [9]:
import logging

logger = logging.getLogger("imprint")

ch.backup(db, chdb)

[worker_id=None] 2023-03-15 15:40:06,474 - confirm.cloud.clickhouse - DEBUG 
Query took 0.07324099540710449 seconds

            select * from information_schema.tables
                where table_schema = 'c0adc07c960a4a809dedf9310f8a0363'
                    and table_name = 'results'
             
[worker_id=None] 2023-03-15 15:40:06,590 - confirm.cloud.clickhouse - DEBUG 
Command took 0.11508703231811523 seconds
DROP TABLE results
[worker_id=None] 2023-03-15 15:40:07,006 - confirm.cloud.clickhouse - DEBUG 
Command took 0.4156670570373535 seconds

        CREATE TABLE results (id Nullable(UInt64),active Nullable(Boolean),parent_id Nullable(UInt64),theta0 Nullable(Float64),radii0 Nullable(Float64),null_truth0 Nullable(Boolean),K Nullable(Int64),coordination_id Nullable(Int64),step_id Nullable(Int64),zone_id Nullable(UInt32),packet_id Nullable(Int64),creator_id Nullable(Int64),creation_time Nullable(Float64),processor_id Nullable(Int64),processing_time Nullable(Float64),eligible Nulla

In [21]:

db2 = ada.DuckDBTiles.connect()
ch.restore(db2, chdb)

[worker_id=None] 2023-03-15 15:43:41,837 - confirm.cloud.clickhouse - DEBUG 
Query took 0.07002592086791992 seconds

            select * from information_schema.tables
                where table_schema = 'c0adc07c960a4a809dedf9310f8a0363'
                    and table_name = 'results'
             
[worker_id=None] 2023-03-15 15:43:41,868 - confirm.cloud.clickhouse - DEBUG 
Query took 0.030493974685668945 seconds
select * from results
[worker_id=None] 2023-03-15 15:43:41,904 - confirm.cloud.clickhouse - DEBUG 
Query took 0.022923946380615234 seconds

            select * from information_schema.tables
                where table_schema = 'c0adc07c960a4a809dedf9310f8a0363'
                    and table_name = 'tiles'
             
[worker_id=None] 2023-03-15 15:43:41,925 - confirm.cloud.clickhouse - DEBUG 
Query took 0.02032303810119629 seconds
select * from tiles
[worker_id=None] 2023-03-15 15:43:41,954 - confirm.cloud.clickhouse - DEBUG 
Query took 0.02512383460998535 seconds

     

In [22]:
import pandas as pd
for table in ch.all_tables:
    if not db.does_table_exist(table):
        continue
    orig = db.con.query(f'select * from {table}').df()
    restored = db2.con.query(f'select * from {table}').df()
    pd.testing.assert_frame_equal(orig, restored)

In [26]:
log_df = db.con.query('select * from logs').df()
log_df.head()['t']

0   2023-03-15 15:35:28.419526
1   2023-03-15 15:35:28.427826
2   2023-03-15 15:35:28.428231
3   2023-03-15 15:35:28.428533
4   2023-03-15 15:35:28.431420
Name: t, dtype: datetime64[ns]

In [24]:
log_df2 = db2.con.query('select * from logs').df()
log_df2.head()['t']

0   2023-03-15 15:35:28.419526
1   2023-03-15 15:35:28.427826
2   2023-03-15 15:35:28.428231
3   2023-03-15 15:35:28.428533
4   2023-03-15 15:35:28.431420
Name: t, dtype: datetime64[ns]